# Conversions

Convert between core hypergraph types by dropping direction, time, or layer information.
Weights are summed and metadata is merged when duplicate hyperedges appear.


In [1]:
import hypergraphx as hgx


## Directed to Hypergraph


In [2]:
dh = hgx.DirectedHypergraph(weighted=True)
dh.add_edge(((0, 1), (2,)), weight=2.0, metadata={"kind": "a"})
dh.add_edge(((2,), (0, 1)), weight=3.0, metadata={"kind": "b"})
hg = dh.to_hypergraph()
hg.get_edges(), hg.get_weight((0, 1, 2)), hg.get_edge_metadata((0, 1, 2))


([(0, 1, 2)], 5.0, {'kind': ['a', 'b']})

## Temporal to Hypergraph


In [3]:
th = hgx.TemporalHypergraph(weighted=True)
th.add_edge((0, 1), time=1, weight=1.0, metadata={"t": 1})
th.add_edge((0, 1), time=2, weight=2.0, metadata={"t": 2})
th.add_edge((1, 2), time=3, weight=3.0)
hg = th.to_hypergraph()
hg.get_edges(), hg.get_weight((0, 1)), hg.get_edge_metadata((0, 1))


([(0, 1), (1, 2)], 3.0, {'t': [1, 2]})

## Multiplex to Hypergraph


In [4]:
mx = hgx.MultiplexHypergraph(weighted=True)
mx.add_edge((0, 1), layer="L1", weight=1.0, metadata={"layer": "L1"})
mx.add_edge((0, 1), layer="L2", weight=2.0, metadata={"layer": "L2"})
hg = mx.to_hypergraph()
hg.get_edges(), hg.get_weight((0, 1)), hg.get_edge_metadata((0, 1))


([(0, 1)], 3.0, {'layer': ['L1', 'L2']})